In [ ]:
!pip install pillow
# !pip install cv2

In [ ]:
!pip install ipywidgets

In [ ]:
import os
import csv
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image

class ImageTagger:
    def __init__(self):
        self.base_dir = 'train_data'
        self.conditions = [d for d in os.listdir(self.base_dir) if os.path.isdir(os.path.join(self.base_dir, d))]
        self.images = self.get_all_images()
        self.body_parts = ["Head", "Neck", "Chest", "Back", "Arms", "Hands", "Legs", "Feet", "Other"]
        self.current_index = 0
        self.tags = {}
        self.last_seen_index = -1
        self.output = widgets.Output()

    def get_all_images(self):
        images = []
        for condition in self.conditions:
            condition_dir = os.path.join(self.base_dir, condition)
            for image in os.listdir(condition_dir):
                if image.lower().endswith(('.png', '.jpg', '.jpeg')):
                    images.append((os.path.join(condition_dir, image), condition))
        return images

    def show_image(self):
        image_path, condition = self.images[self.current_index]
        img = Image.open(image_path)
        img.thumbnail((400, 400))  # Resize image for display
        return widgets.Image(value=self.pil_to_bytes(img), format='png')

    def pil_to_bytes(self, img):
        import io
        buf = io.BytesIO()
        img.save(buf, format='PNG')
        return buf.getvalue()

    def on_button_clicked(self, b):
        if b.description == 'Skip':
            self.current_index = (self.current_index + 1) % len(self.images)
        elif b.description == 'Quit':
            self.save_to_csv()
            return

        self.last_seen_index = max(self.last_seen_index, self.current_index)
        self.update_display()

    def on_tag_selected(self, change):
        image_path, _ = self.images[self.current_index]
        self.tags[image_path] = change['new']
        self.current_index = (self.current_index + 1) % len(self.images)
        self.last_seen_index = max(self.last_seen_index, self.current_index)
        self.update_display()

    def update_display(self):
        with self.output:
            clear_output(wait=True)
            image_widget = self.show_image()
            _, condition = self.images[self.current_index]

            radio_button = widgets.RadioButtons(
                options=self.body_parts,
                description='Body Part:',
                disabled=False
            )
            radio_button.observe(self.on_tag_selected, names='value')

            skip_button = widgets.Button(description="Skip")
            quit_button = widgets.Button(description="Quit")
            skip_button.on_click(self.on_button_clicked)
            quit_button.on_click(self.on_button_clicked)

            display(widgets.VBox([
                widgets.HTML(f"<h3>Image {self.current_index + 1} of {len(self.images)}</h3>"),
                widgets.HTML(f"<h4>Condition: {condition}</h4>"),
                image_widget,
                radio_button,
                widgets.HBox([skip_button, quit_button])
            ]))

    def run(self):
        display(self.output)
        self.update_display()

    def save_to_csv(self):
        with open('image_tags.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Image Path", "Condition", "Body Part"])
            for i, (image_path, condition) in enumerate(self.images):
                if i <= self.last_seen_index and image_path in self.tags:
                    writer.writerow([image_path, condition, self.tags[image_path]])
        print(f"Tags saved to image_tags.csv for seen images.")

# Run the tagger
tagger = ImageTagger()
tagger.run()